In [23]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\D.Ivanovas\.cache\kagglehub\datasets\clmentbisaillon\fake-and-real-news-dataset\versions\1


In [24]:
import pandas as pd
import os 

fake_df = pd.read_csv(os.path.join(path, 'fake.csv'))
true_df = pd.read_csv(os.path.join(path, 'true.csv'))

fake_df["label"]=0
true_df["label"]=1

joined_df = pd.concat([fake_df, true_df], axis=0, ignore_index=True) # sujungiam sulablintus datasetus
joined_df = joined_df.sample(frac=1).reset_index(drop=True) # sushufflinam dataseta


# Features

In [26]:
from spellchecker import SpellChecker
import spacy
import swifter
import re

spell = SpellChecker()
nlp = spacy.load('en_core_web_sm')

def extract_features(text):
    doc = nlp(text)
    word_count = len(text.split())
    stopword_count = sum(1 for token in doc if token.is_stop)
    punct_count = sum(1 for token in doc if token.is_punct)
    ent_count = len(doc.ents)
    misspelled = spell.unknown(text)  # Using `spell.unknown` to get misspelled words
    wierd_symbols_count = len(misspelled)
    
    return word_count, stopword_count, punct_count, wierd_symbols_count, ent_count

joined_df[['word_count', 'stopword_count', 'punct_count', 'wierd_symbols_count', 'ent_count']] = \
    joined_df['text'].swifter.apply(lambda text: extract_features(text)).apply(pd.Series)


Pandas Apply:   0%|          | 0/44898 [00:00<?, ?it/s]

# Text processing

In [ ]:


# '\xad' soft hyphen | \u200e  left to right symbol | \u200a Unicode Character “ ” | \u200b Zero-width space
# \u200f Right-to-Left Mark
unwanted_symbols = ['–', '—', '‘', '“', '”', '•', '…', '☑', '➡', 'ツ',  '¯','°', '´', '¿', '\xad', '\u200e', '\u200a', '\u200b', '\u200f']

def process_text(text):
    # Turkey’s -> Turkey's
    text = re.sub(r'(?<=\w)’(?=s)', "'", text)
    # replace all whitespaces, including unicode spaces and tabs with regular space
    pattern = '[' + re.escape(''.join(unwanted_symbols)) + ']'  # Escapes special regex characters
    text = re.sub(pattern, '', text)    
    text = re.sub(r'\s', ' ', text)
    doc = nlp(text)
    sent = []
    # check for stop words and punctuation
    for token in doc:
        if not token.is_punct and not token.is_stop:
            sent.append(token.text)
    
    return ' '.join(sent)

# use swifter for faster .apply()
joined_df['processed_text'] = joined_df['text'].apply(process_text)

joined_df.to_csv("copy.csv")

Pandas Apply:   0%|          | 0/44898 [00:00<?, ?it/s]

In [14]:
import sys
sys.path.append(r'C:\Users\D.Ivanovas\Desktop\uni\Fake-News')

In [15]:
from paths import DATA_DIR
import shutil

shutil.move(str(path), str(DATA_DIR))

'C:\\Users\\D.Ivanovas\\Desktop\\uni\\Fake-News\\data'